In [1]:
!nvidia-smi

Tue Nov 22 10:14:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 460.27.04    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 30%   26C    P8     3W / 250W |   6761MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 30%   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import numpy as np
# from tqdm.auto import tqdm
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
print(torch.cuda.device_count())

cuda
1


In [4]:
CONFIG = {
    'EPOCHS': 100,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':512,
    'SEED':101
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG['SEED'])

In [6]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Preprocessing

In [7]:
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR' , 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

In [8]:
train = train.fillna(0)
test = test.fillna(0)

In [9]:
all_X = train.drop(['ID', 'Y_LABEL'], axis = 1)
all_y = train['Y_LABEL']

test = test.drop(['ID'], axis = 1)

train_X, val_X, train_y, val_y = train_test_split(all_X, all_y, test_size=0.2, random_state=CONFIG['SEED']) #, stratify=all_y)

In [10]:
# ---------------------------
# Handling Numerical
# ---------------------------
def get_values(value):
    return value.values.reshape(-1, 1)

for col in train_X.columns:
    if col not in categorical_features:
        scaler = StandardScaler()
        train_X[col] = scaler.fit_transform(get_values(train_X[col]))
        val_X[col] = scaler.transform(get_values(val_X[col]))
        if col in test.columns:
            test[col] = scaler.transform(get_values(test[col]))
            
            
# ---------------------------
# Handling Categorical
# ---------------------------
le = LabelEncoder()
for col in categorical_features:    
    train_X[col] = le.fit_transform(train_X[col])
    val_X[col] = le.transform(val_X[col])
    if col in test.columns:
        test[col] = le.transform(test[col])

# Large Model (teacher)

In [11]:
# ---------------------------
# Custom Dataset
# ---------------------------
class CustomDataset(Dataset):
    def __init__(self, data_X, data_y, distillation=False):
        super(CustomDataset, self).__init__()
        self.data_X = data_X
        self.data_y = data_y
        self.distillation = distillation
        
    def __len__(self):
        return len(self.data_X)
    
    def __getitem__(self, index):
        if self.distillation:
            #------------------------------------
            # When Learing with distillation
            #------------------------------------
            teacher_X = torch.Tensor(self.data_X.iloc[index])
            student_X = torch.Tensor(self.data_X[test_stage_features].iloc[index])
            label_y   = torch.Tensor([self.data_y.iloc[index]])
            return teacher_X.to(device), \
                   student_X.to(device), \
                   label_y.to(device)
        else:
            #------------------------------------
            # When Learing with Normal Data
            #------------------------------------
            if self.data_y is None: # for submission dataset
                test_X = torch.Tensor(self.data_X.iloc[index])
                return test_X.to(device)
            
            else:
                teacher_X = torch.Tensor(self.data_X.iloc[index])
                label_y   = torch.Tensor([self.data_y.iloc[index]])
                return teacher_X.to(device), \
                       label_y.to(device)

In [12]:
# ---------------------------
# Dataset for Teacher Model
# ---------------------------
train_dataset = CustomDataset(train_X, train_y, False)
val_dataset = CustomDataset(val_X, val_y, False)

# ---------------------------
# Dataloader for Teacher Model
# ---------------------------
train_loader = DataLoader(train_dataset, batch_size = CONFIG['BATCH_SIZE'], shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size = CONFIG['BATCH_SIZE'], shuffle=False)

In [13]:
val_dataset[0]

(tensor([ 0.0000e+00,  2.7493e-04,  8.0000e+00, -3.9161e-01, -3.4112e-01,
         -1.4855e-01, -1.0477e-01,  2.4425e+00, -3.0059e-01, -3.7119e-02,
          8.3046e-01, -6.7052e-02, -1.2607e-01, -1.1260e-01, -2.6032e-01,
          4.9347e-01,  1.1905e+00, -3.6082e-01,  7.6335e-01,  1.1156e+00,
          1.3627e+00, -3.6135e-01, -1.0803e-01, -3.9834e-02, -7.1962e-02,
         -1.0763e-01, -1.9431e-01, -3.3823e-01, -4.0253e-01, -4.1215e-04,
         -1.9460e-01,  8.8570e-02, -1.6250e-01, -2.6765e-01, -8.3927e-01,
         -1.9846e-01, -1.6062e-01, -2.4240e-01,  5.1207e-01, -1.0670e-01,
         -7.8102e-02, -1.0621e-01, -7.1571e-02, -1.1962e-01, -1.2277e-01,
         -1.3725e-01, -1.5107e-01, -1.9664e-01, -1.0860e-01,  1.6066e+00,
         -1.4155e-01,  9.7597e-01], device='cuda:0'),
 tensor([1.], device='cuda:0'))

In [14]:
# ---------------------------
# Teacher Model
# ---------------------------
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=52, out_features=256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Linear(in_features=256, out_features=512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Linear(in_features=512, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Linear(in_features=1024, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Linear(in_features=1024, out_features=512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Linear(in_features=512, out_features=256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [15]:
#--------------------------------------------------------------------------------
# Metrics
#--------------------------------------------------------------------------------
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")


#--------------------------------------------------------------------------------
# Training Teacher
#--------------------------------------------------------------------------------
def teacher_train(model, optimizer, train_loader, val_loader, scheduler):
    model.to(device)

    best_score = 0
    best_model = None
    criterion = nn.BCELoss().to(device)

    for epoch in range(CONFIG["EPOCHS"]):
        train_loss = []
        
        # ACTIVATE TRAINING MODE
        # --- normalisation layers1 use per-batch statistics
        # --- activates Dropout layers2
        model.train()
    
        for i,(X, y) in tqdm(enumerate(train_loader)):
            
            # ZERO GRADIENT
            optimizer.zero_grad()
            
            # FORWARD
            y_pred = model(X)
            loss = criterion(y_pred, y)
            
            # BACKWARD
            loss.backward()
            
            # UPDATE
            optimizer.step()

            train_loss.append(loss.item())

        val_loss, val_score = validation_teacher(model, val_loader, criterion)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model 


#--------------------------------------------------------------------------------
# Validation Teacher
#--------------------------------------------------------------------------------
def validation_teacher(model, val_loader, criterion):
    # ACTIVATE EVALUATION MODE
    # --- normalisation layers use running statistics
    # --- de-activates Dropout layers
    model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = 0.4
    
    with torch.no_grad():
        for i,(X, y) in tqdm(enumerate(val_loader)):
            
            model_pred = model(X)
            
            loss = criterion(model_pred,y)
            val_loss.append(loss.item())
            
            model_pred = model_pred.squeeze(1)#.to('cpu')  
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_f1 = competition_metric(true_labels, pred_labels)
    return val_loss, val_f1

In [16]:
#--------------------------------------------------------------------------------
# Training Process
#--------------------------------------------------------------------------------
model = Teacher()
optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, threshold_mode='abs',min_lr=1e-6, verbose=True)

teacher_model = teacher_train(model, optimizer, train_loader, val_loader, scheduler)

23it [00:03,  6.43it/s]
6it [00:00,  7.27it/s]


Epoch [0], Train Loss : [0.35824] Val Loss : [0.21550] Val F1 Score : [0.75216]


23it [00:03,  6.64it/s]
6it [00:00,  6.99it/s]


Epoch [1], Train Loss : [0.22545] Val Loss : [0.21751] Val F1 Score : [0.73621]


23it [00:03,  6.63it/s]
6it [00:00,  6.91it/s]


...

Epoch [97], Train Loss : [0.10353] Val Loss : [0.17731] Val F1 Score : [0.80279]


23it [00:03,  6.60it/s]
6it [00:00,  7.33it/s]


Epoch [98], Train Loss : [0.11831] Val Loss : [0.17421] Val F1 Score : [0.80704]


23it [00:03,  6.73it/s]
6it [00:00,  7.33it/s]

Epoch [99], Train Loss : [0.10921] Val Loss : [0.17467] Val F1 Score : [0.81302]


# Small Model (student)

In [17]:
# ---------------------------
# Student Model
# ---------------------------
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=18, out_features=128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Linear(in_features=128, out_features=256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Linear(in_features=256, out_features=256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Linear(in_features=256, out_features=128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Linear(in_features=128, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [18]:
# # ---------------------------
# # Distillation Loss with Temperature
# # ---------------------------

# def distillation(student_logits, labels, teacher_logits, T,alpha):
#     distillation_loss = nn.BCELoss()(student_logits/T, teacher_logits/T)
#     student_loss = nn.BCELoss()(student_logits, labels.reshape(-1, 1))
#     return (1.-alpha) * student_loss +  distillation_loss * (T*T * 2.0 * alpha)

# def distill_loss(output, target, teacher_output, loss_fn=distillation, opt=optimizer):
#     loss_b = loss_fn(output, target, teacher_output, T=20.0, alpha=0.7)

#     if opt is not None:
#         # BACKWARD
#         loss_b.backward()
        
#         # UPDATE
#         opt.step()

#     return loss_b.item()

In [19]:
# ---------------------------
# Distillation Loss without Temperature
# ---------------------------

def distillation(student_logits, labels, teacher_logits, alpha):
    distillation_loss = nn.BCELoss()(student_logits, teacher_logits)
    student_loss = nn.BCELoss()(student_logits, labels.reshape(-1, 1))
    return (1.-alpha) * student_loss +  distillation_loss * alpha

def distill_loss(output, target, teacher_output, loss_fn=distillation, opt=optimizer):
    loss_b = loss_fn(output, target, teacher_output, alpha=0.7)

    if opt is not None:
        # BACKWARD
        loss_b.backward()
        
        # UPDATE
        opt.step()

    return loss_b.item()

In [20]:
#--------------------------------------------------------------------------------
# Training Student
#--------------------------------------------------------------------------------

def student_train(s_model, t_model, optimizer, train_loader, val_loader, scheduler, device):
    
    s_model.to(device)
    t_model.to(device)
    
    best_score = 0
    best_model = None

    for epoch in range(CONFIG["EPOCHS"]):
        train_loss = []
        s_model.train()
        t_model.eval()
        
        for X_t, X_s, y in tqdm(train_loader):
            
            # ZERO GRADIENT
            optimizer.zero_grad()
            
            # FORWARD
            y_pred_student = s_model(X_s)
            with torch.no_grad():
                y_pred_teacher = t_model(X_t)
                
            loss_b = distill_loss(y_pred_student, y, y_pred_teacher, loss_fn=distillation, opt=optimizer)
            
                
            train_loss.append(loss_b)

        val_loss, val_score = validation_student(s_model, t_model, val_loader, distill_loss, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = s_model
            best_score = val_score
        
    return best_model


#--------------------------------------------------------------------------------
# Validation Student
#--------------------------------------------------------------------------------

def validation_student(s_model, t_model, val_loader, criterion, device):
    s_model.eval()
    t_model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = 0.35
    
    with torch.no_grad():
        for X_t, X_s, y in tqdm(val_loader):
            
            model_pred     = s_model(X_s)
            teacher_output = t_model(X_t)
            
            loss_b = distill_loss(model_pred, y, teacher_output, loss_fn=distillation, opt=None)
            val_loss.append(loss_b)
            
            model_pred = model_pred.squeeze(1)#.to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_f1 = competition_metric(true_labels, pred_labels)
    return val_loss, val_f1    

In [21]:
# ---------------------------
# Dataset for Student Model (Distillation True)
# ---------------------------
train_dataset = CustomDataset(train_X, train_y, True)
val_dataset = CustomDataset(val_X, val_y, True)

# ---------------------------
# Dataloader for Student Model (Distillation True)
# ---------------------------
train_loader = DataLoader(train_dataset, batch_size = CONFIG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = CONFIG['BATCH_SIZE'], shuffle=False)

In [22]:
#--------------------------------------------------------------------------------
# Training Process
#--------------------------------------------------------------------------------

student_model = Student()
# student_model.eval()
optimizer = torch.optim.Adam(student_model.parameters(), lr=CONFIG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, threshold_mode='abs',min_lr=1e-6, verbose=True)

best_student_model = student_train(student_model, teacher_model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 6/6 [00:02<00:00,  2.06it/s]


Epoch [0], Train Loss : [0.46893] Val Loss : [0.45384] Val F1 Score : [0.49809]


100%|██████████| 6/6 [00:02<00:00,  2.05it/s]


Epoch [1], Train Loss : [0.33719] Val Loss : [0.31709] Val F1 Score : [0.48479]


100%|██████████| 6/6 [00:02<00:00,  2.06it/s]


...

Epoch [97], Train Loss : [0.26236] Val Loss : [0.32366] Val F1 Score : [0.54471]


100%|██████████| 6/6 [00:02<00:00,  2.05it/s]


Epoch [98], Train Loss : [0.26489] Val Loss : [0.32224] Val F1 Score : [0.54102]
Epoch    99: reducing learning rate of group 0 to 7.8125e-06.


100%|██████████| 6/6 [00:02<00:00,  2.06it/s]

Epoch [99], Train Loss : [0.26313] Val Loss : [0.32422] Val F1 Score : [0.54323]


# Choose Inference Threshold

In [31]:
def choose_threshold(model, val_loader, device):
    model.to(device)
    model.eval()
    
    thresholds = [0.1, 0.175, 0.195, 0.198 ,0.2,  0.21, 0.225, 0.23, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
    pred_labels = []
    true_labels = []
    
    best_score = 0
    best_thr = None
    with torch.no_grad():
        for _, x_s, y in tqdm(iter(val_loader)):
            
            model_pred = model(x_s)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        for threshold in thresholds:
            pred_labels_thr = np.where(np.array(pred_labels) > threshold, 1, 0)
            score_thr = competition_metric(true_labels, pred_labels_thr)
            if best_score < score_thr:
                best_score = score_thr
                best_thr = threshold
    return best_thr, best_score

In [32]:
best_threshold, best_score = choose_threshold(best_student_model, val_loader, device)
print(f'Best Threshold : [{best_threshold}], Score : [{best_score:.5f}]')

100%|██████████| 6/6 [00:02<00:00,  2.02it/s]


Best Threshold : [0.198], Score : [0.55828]


# Inference

In [ ]:
test_datasets = CustomDataset(test, None, False)
test_loaders = DataLoader(test_datasets, batch_size = CONFIG['BATCH_SIZE'], shuffle=False)

In [ ]:
def inference(model, test_loader, threshold, device):
    model.to(device)
    model.eval()
    
    test_predict = []
    with torch.no_grad():
        for x in tqdm(test_loader):
            x = x.float().to(device)
            model_pred = model(x)

            model_pred = model_pred.squeeze(1).to('cpu')
            test_predict += model_pred
        
    test_predict = np.where(np.array(test_predict) > threshold, 1, 0)
    print('Done.')
    return test_predict

In [ ]:
preds = inference(best_student_model, test_loaders, best_threshold, device)

# Submit

In [ ]:
submit = pd.read_csv('data/sample_submission.csv')
submit['Y_LABEL'] = preds
submit.head()

In [ ]:
submit.to_csv('./submit_GELU_epoch100_alpha0.7_0.4_0.35.csv', index=False)